In [1]:
import requests as req
from bs4 import BeautifulSoup as bs
from datetime import datetime
import csv
import re
import string

hades = { 'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36'}


In [2]:
def clear_content(text):
    text = text.lower()
    text = re.sub(r"#[A-Za-z0-9_]+", "", text)
    text = re.sub(r'http\S+', '', text)  # remove links
    text = re.sub(r'[0-9]+', '', text)  # remove numbers
    text = text.replace('\n', ' ')  # remove new lines
    text = text.translate(str.maketrans('', '', string.punctuation))  # remove punctuation
    text = text.strip()  # trim spaces
    text = text.replace("¬©", '')  # remove special symbols
    return text

In [3]:
def clean_additional_content(content):
    # Remove specific phrases
    content = content.replace("jakarta cnbc indonesia", "")
    content = content.replace("jakarta cnbcÂ indonesiaÂ", "")
    content = content.replace("advertisement", "")
    content = content.replace("scroll to resume content", "")
    content = content.replace("Â", "")

    # Remove extra spaces
    content = ' '.join(content.split())

    # Remove empty lines
    content = '\n'.join(line.strip() for line in content.splitlines() if line.strip())

    return content

In [4]:
def scrape_cnbc(hal):
    global hades
    a = 1
    csv_file = 'cnbc_result.csv'

    # Write the header to the CSV file
    with open(csv_file, 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file, delimiter=',')
        writer.writerow(['Headline', 'Date', 'Link', 'Content'])

    for page in range(1, hal+1):
        url_cnbc = f"https://www.cnbcindonesia.com/search?query=bea%20cukai&p={page}&kanal=&tipe=artikel&date="
        ge = req.get(url_cnbc, headers=hades).text
        sop = bs(ge, 'lxml')
        ul = sop.find('ul', class_='list media_rows middle thumb terbaru gtm_indeks_feed')
        if ul is None:
            continue
        li = ul.find_all('li')
        for x in li:
            link_art = x.find('article').find('a')['href']
            ge_ = req.get(link_art, headers=hades).text
            sop_ = bs(ge_, 'lxml')
            art_div = sop_.find('div', class_='lm_content mt10')
            if art_div is None:
                continue
            art = art_div.find('article')
            if art is None:
                continue
            headline = art.find('h1').text
            date_text = art.find('div', class_='date').text
            date = datetime.strptime(date_text, '%d %B %Y %H:%M')
            warp = art.find_all('div', class_='detail_text')
            for w in warp:
                paragraphs = w.find_all('p')
                content = ''.join(p.text for p in paragraphs)
                content = clear_content(content)  # clean the content
                content = clean_additional_content(content)  # clean additional content
                print(f'done[{a}] > {headline[0:10]}')
                a += 1
                with open(csv_file, 'a', newline='', encoding='utf-8') as file:
                    writer = csv.writer(file, delimiter=',')
                    writer.writerow([headline, date, link_art, content])